In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/브런치추천/'

/content/drive/MyDrive/브런치추천


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np

In [ ]:
metadata = pd.read_json('processed_data/metadata_topic.json')

In [ ]:
metadata.head(1)

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id,sentence,topic_num_50,topic_num_100,topic_num_20
0,28849,@elang8151,출간제의 받았습니다.,"[메일, 출판사]",https://brunch.co.kr/@elang8151/229,미안하지만 아직 학생입니다.,1539405177000,229,@elang8151_229,출간제의 받았습니다. 메일 출판사 미안하지만 아직 학생입니다.,-1,-1,-1


# Doc2Vec

In [ ]:
documents = metadata['sentence'].tolist()
ids = metadata['user_id'].tolist()

TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in zip(documents,ids)]

model = Doc2Vec(TRAIN_documents, vector_size=300, window=5, epochs=10, min_count=0, workers=4, verbose=True)

In [ ]:
sentence_embedding = {}

In [ ]:
for doc,id in zip(documents,ids):
  if id not in sentence_embedding: sentence_embedding[id] = []
  sentence_embedding[id].append(model.infer_vector(doc).tolist())

In [ ]:
for id in sentence_embedding.keys():
  sentence_embedding[id] = np.stack(sentence_embedding[id],axis=0)
  sentence_embedding[id] = np.mean(sentence_embedding[id],axis=0).reshape(1,-1)

In [ ]:
model = {}
for id in metadata['id'].unique():
  model[id] = sentence_embedding[id.split('_')[0]]

In [ ]:
with open('processed_data/doc2vec.pkl','wb') as f:
  pickle.dump(model,f)